# Part 2: Data Cleaning

    First we will separate categorical and numeric column
    
    Then we will convert some categorical column to numerical column and vice-a-versa
    
    Then prepare the data for final analysis
    
    

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#adding the path of the src dir to the notebook path
import sys
sys.path.insert(0, '/Users/manish/Documents/Projects/data_science/Lending_club_case_study/src')

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 500)

In [2]:
path = '/Users/manish/Documents/Projects/data_science/Lending_club_case_study/data/clean_data/loan_cleaned.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
0,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,...,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,May-16,0.0
1,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,...,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-13,0.0
2,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,...,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,May-16,0.0
3,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,...,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,Apr-16,0.0
4,5000,5000,5000.0,36 months,7.90%,156.46,A,A4,3 years,RENT,...,5632.21,5000.00,632.21,0.00,0.00,0.00,Jan-15,161.03,Jan-16,0.0


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36737 entries, 0 to 36736
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_amnt                36737 non-null  int64  
 1   funded_amnt              36737 non-null  int64  
 2   funded_amnt_inv          36737 non-null  float64
 3   term                     36737 non-null  object 
 4   int_rate                 36737 non-null  object 
 5   installment              36737 non-null  float64
 6   grade                    36737 non-null  object 
 7   sub_grade                36737 non-null  object 
 8   emp_length               36737 non-null  object 
 9   home_ownership           36737 non-null  object 
 10  annual_inc               36737 non-null  float64
 11  verification_status      36737 non-null  object 
 12  issue_d                  36737 non-null  object 
 13  loan_status              36737 non-null  object 
 14  purpose               

In [6]:
from functions import *

In [7]:
#getting the unique values in each column
get_null_unique(df, df.columns)

loan_amnt :  [ 5000  2500  2400 10000  7000  3000  5600  5375  6500 12000]
Unique:  855
Null per_cent:  0.0
------------------------------------------------------------
funded_amnt :  [ 5000  2500  2400 10000  7000  3000  5600  5375  6500 12000]
Unique:  1009
Null per_cent:  0.0
------------------------------------------------------------
funded_amnt_inv :  [ 4975.  2500.  2400. 10000.  5000.  7000.  3000.  5600.  5350.  6500.]
Unique:  7596
Null per_cent:  0.0
------------------------------------------------------------
term :  [' 36 months' ' 60 months']
Unique:  2
Null per_cent:  0.0
------------------------------------------------------------
int_rate :  ['10.65%' '15.27%' '15.96%' '13.49%' '7.90%' '18.64%' '21.28%' '12.69%'
 '14.65%' '9.91%']
Unique:  336
Null per_cent:  0.0
------------------------------------------------------------
installment :  [162.87  59.83  84.33 339.31 156.46 170.08 109.43 152.39 121.45 153.45]
Unique:  14369
Null per_cent:  0.0
--------------------------

### Actions to be done to respective columns

    loan_amnt, num, Bins of 10
    funded_amnt_inv, num Bins of 10
    funded_amnt, num, Bis of 10
    term, cat, no action
    int_rate, cat, remove trailing % and convert to num and bins of 10
    installment, num, Bins of 10
    grade, cat, no action
    sub_grade, cat, no_action
    emp_length, cat, convert to <=1, 1-5, 5-10 and 10+
    home_ownership, cat, no action
    annual_inc, num, Bins of 10
    verifiaction_status, cat, no action
    issue_d, cat, get months
    loan_status, cat, no action
    purpose, cat, no action
    dti, num, Bins of 10
    delinq_2yrs, num, no change, actually treat as categorical
    earliest_cr_line, cat, get year
    inq_last_6mths, num, no action, but treat as categorical
    open_acc, num, bins of 10
    pub_rec, num, no action, treat as categorical
    revol_bal, num, Bins of 10
    revol_util, cat, remove trailing % and convert to num and in bins of 10
    total_acc, num, bins of 10
    total_payment, num, bins of 10
    total_pymnt_inv, num, bins of 10 or drop as redundant to total_payment
    total_rec_prncp, num, bins of 10 or a redundant column can be dropped
    total_rec_int, num, bins of 10
    total_rec_late_fee, num, bins of 10
    recoveries, num, bins of 10
    collection_rec_fee, num, bins of 10 or can be dropped as seems redundant
    last_pymnt_d, cat, convert to month
    last_pymnt_amnt, num, bins of 10
    last_credit_pull_d, cat, convert to year and month
    pub_rec_bankruptcies, cat, no action

def remove_trailing(df, col = None):
    
    
    return df[col].applymap(lambda x: float(x.rstrip('%')))

    

In [8]:
#resolving interest rate and revol_util
df[['int_rate', 'revol_util']] = remove_trailing(df,['int_rate', 'revol_util'] )
df[['int_rate', 'revol_util']].head()

,int_rate,revol_util
0,10.65,83.7
1,15.27,9.4
2,15.96,98.5
3,13.49,21.0
4,7.90,28.3


def to_bins(df, col = None):
    
    resolve = []
    
    k = max(df[col])/3
    
    
    for x in df[col]:
        
        if x < k:
            
            z =  'new'
        
        elif x >= k and x <= 2 * k :
            
            z =  'mid'
    
        else:
            z =  'old'
        
        resolve.append(z)
        
    return resolve
    

In [9]:
#converting the emp_length to bins
df.emp_length = df.emp_length.apply(lambda x : int(x.rstrip( '+ years').lstrip('<')))
df.emp_length = to_bins(df, 'emp_length')

In [10]:
#resolving delinq_2yrs
df.delinq_2yrs.value_counts()

0     32755
1      3058
2       633
3       197
4        55
5        21
6        10
7         4
8         2
9         1
11        1
Name: delinq_2yrs, dtype: int64

In [11]:
#converting numerical columns to required categorical
convert_to_cat = ['delinq_2yrs', 'inq_last_6mths', 'pub_rec']
df[convert_to_cat] = df[convert_to_cat].applymap(lambda x : str(x))
df[convert_to_cat].head()

,delinq_2yrs,inq_last_6mths,pub_rec
0,0,1,0
1,0,5,0
2,0,2,0
3,0,1,0
4,0,3,0


In [12]:
#not keeping date wise columns 
date_wise = ['issue_d', 'last_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line']

In [13]:
df = df.drop(date_wise, axis = 1)

In [14]:
#saving the processed dataframe to analysis file
#df.to_csv("loan_for_analysis.csv", sep = ',', index = False)

### Lets move to analysis file